In [6]:
import pandas as pd

custom_na_values = ["Not Provided"]

data_csv = pd.read_csv(
    "data/IND_TariffLine.csv",
    na_values=custom_na_values,
    keep_default_na=False,
    dtype=str,
).fillna("")
data_csv

NomenCode ReporterISO3 ProductCode  \
0            H1          IND     7122000   
1            H1          IND     7131000   
2            H1          IND     7133901   
3            H1          IND     8042002   
4            H1          IND     8045002   
...         ...          ...         ...   
55341        H3          IND    34011930   
55342        H3          IND    34011941   
55343        H3          IND    34011942   
55344        H3          IND    34011990   
55345        H3          IND    34012000   

                                             Description Source Match  \
0                                                           IDB     *   
1                                                           IDB     *   
2                                                           IDB     *   
3                                                           IDB     *   
4                                                           IDB     *   
...                                                  ...    ...   ...   
55341  Tablets and cakes: Household and laundry soaps...    IDB     X   
55342                                    Household soaps    IDB     X   
55343                                      Laundry soaps    IDB     X   
55344                                              Other    IDB     X   
55345                                Soap in other forms    IDB     X   

      ProductCode_IDB ProductCode_TRN  \
0               71220           71220   
1               71310           71310   
2               71339           71339   
3               80420           80420   
4               80450           80450   
...               ...             ...   
55341        34011930        34011930   
55342        34011941        34011941   
55343        34011942        34011942   
55344        34011990        34011990   
55345        34012000        34012000   

                                  ProductDescription_IDB  \
0                                                          
1                                                          
2                                                          
3                                                          
4                                                          
...                                                  ...   
55341  Tablets and cakes: Household and laundry soaps...   
55342                                    Household soaps   
55343                                      Laundry soaps   
55344                                              Other   
55345                                Soap in other forms   

                                  ProductDescription_TRN  \
0                                                          
1                                                          
2                                                          
3                                                          
4                                                          
...                                                  ...   
55341  Tablets and cakes: <br> Household and laundry ...   
55342  Soap and organic surface-active products and p...   
55343  Soap and organic surface-active products and p...   
55344  Soap and organic surface-active products and p...   
55345  Soap in the form of flakes, granules, powder, ...   

                                  ProductDescription_HS6  
0                                                 Onions  
1                                   Peas (Pisum sativum)  
2                                                  Other  
3                                                   Figs  
4                        Guavas, mangoes and mangosteens  
...                                                  ...  
55341  Soap & organic surface-active products & prepa...  
55342  Soap & organic surface-active products & prepa...  
55343  Soap & organic surface-active products & prepa...  
55344  Soap & organic surface-active products & prepa...  
55345   Soap in 

In [9]:
data_csv.tail(10).to_csv('test.csv', index=False)

In [34]:
import nltk

nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [70]:
import re
from collections import defaultdict

import pandas as pd
from bs4 import BeautifulSoup
from textblob import TextBlob, Word


def combine_descriptions(row):
    idb = row["ProductDescription_IDB"]
    hs6 = row["ProductDescription_HS6"]

    soup = lambda desc: BeautifulSoup(desc, "html.parser").get_text()

    desc = ""

    def clean_excl_incl(desc):
        # Handle exclusions
        if "excl. " in desc:
            desc = desc.replace("excl. ", "excluding ")
        if "incl. " in desc:
            desc = desc.replace("incl. ", "including ")
        return desc

    desc = " >> ".join(
        [
            y
            for y in pd.Series(
                [
                    str(clean_excl_incl(soup(hs6))) if pd.notna(hs6) else None,
                    str(clean_excl_incl(soup(idb))) if pd.notna(idb) else None,
                ]
            )
            .fillna("")
            .tolist()
            if y and y.strip()
        ]
    )

    desc = " >> ".join(
        [
            re.sub(r"[\?\.]+(?=$)", "", str(sentence))
            for sentence in TextBlob(desc).sentences
        ]
    )
    # Further cleanup
    desc = re.sub(r"\s+", " ", desc)  # remove double spaces

    return desc


df = pd.read_csv("test.csv")
df["Cleaned_Description"] = df.apply(combine_descriptions, axis=1)
df

/tmp/ipykernel_247/2670806812.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = lambda desc: BeautifulSoup(desc, "html.parser").get_text()


NomenCode ReporterISO3  ProductCode  \
0        H3          IND     34011120   
1        H3          IND     34011190   
2        H3          IND     34011911   
3        H3          IND     34011919   
4        H3          IND     34011920   
5        H3          IND     34011930   
6        H3          IND     34011941   
7        H3          IND     34011942   
8        H3          IND     34011990   
9        H3          IND     34012000   

                                         Description Source Match  \
0             Shaving soaps other than shaving cream    IDB     X   
1                                              Other    IDB     X   
2                                    Industrial soap    IDB     X   
3                                              Other    IDB     X   
4                          Flakes, chips and powder:    IDB     X   
5  Tablets and cakes: Household and laundry soaps...    IDB     X   
6                                    Household soaps    IDB     X   
7                                      Laundry soaps    IDB     X   
8                                              Other    IDB     X   
9                                Soap in other forms    IDB     X   

   ProductCode_IDB  ProductCode_TRN  \
0         34011120         34011120   
1         34011190         34011190   
2         34011911         34011911   
3         34011919         34011919   
4         34011920         34011920   
5         34011930         34011930   
6         34011941         34011941   
7         34011942         34011942   
8         34011990         34011990   
9         34012000         34012000   

                              ProductDescription_IDB  \
0             Shaving soaps other than shaving cream   
1                                              Other   
2                                    Industrial soap   
3                                              Other   
4                          Flakes, chips and powder:   
5  Tablets and cakes: Household and laundry soaps...   
6                                    Household soaps   
7                                      Laundry soaps   
8                                              Other   
9                                Soap in other forms   

                              ProductDescription_TRN  \
0  Soap and organic surface-active products and p...   
1  Soap and organic surface-active products and p...   
2  Soap and organic surface-active products and p...   
3  Soap and organic surface-active products and p...   
4  Soap and organic surface-active products and p...   
5  Tablets and cakes: <br> Household and laundry ...   
6  Soap and organic surface-active products and p...   
7  Soap and organic surface-active products and p...   
8  Soap and organic surface-active products and p...   
9  Soap in the form of flakes, granules, powder, ...   

                              ProductDescription_HS6  \
0  Soap & organic surface-active products & prepa...   
1  Soap & organic surface-active products & prepa...   
2  Soap & organic surface-active products & prepa...   
3  Soap & organic surface-active products & prepa...   
4  Soap & organic surface-active products & prepa...   
5  Soap & organic surface-active products & prepa...   
6  Soap & organic surface-active products & prepa...   
7  Soap & organic surface-active products & prepa...   
8  Soap & organic surface-active products & prepa...   
9   Soap in other forms (excl. of 3401.11 & 3401.19)   

                                 Cleaned_Description  
0  Soap & organic surface-active products & prepa...  
1  Soap & organic surface-active products & prepa...  
2  Soap & organic surface-active products & prepa...  
3  Soap & organic surface-active products & prepa...  
4  Soap & organic surface-active products & prepa...  
5  Soap & organic surface-active products & prepa...  
6  Soap & organic surface-active products & prepa...  
7  Soap & organic surface-active products & prepa...  
8  Soap & orga

In [71]:
df.to_csv("test1.csv", index=False)